In [1]:
from utils.data_wrapper import DataWrapper
from utils.visualizer import Visualizer

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd
from src.models.baseline.tree_models import XGBoost, LightGBM
from src.models.baseline.baseline_models import LinearModel, KNN, SVM, DecisionTree, RandomForest

In [4]:
df = DataWrapper(df="../data/final_dataset.csv")

../data/final_dataset.csv


In [5]:
df.df

,date,time,classroom_type,classroom_category,room_no,device_code,school_no,grade,measured_t,measured_rh,...,airtc_avg,rh,ws_ms_s_wvt,winddir_d1_wvt,winddir_sd1_wvt,bp_mb_avg,uva_avg,uvb_avg,battv_min,batt24v_min
0,08/02/2023,7:34:34,Container No Insulation,Temporary,1,3,1,3,20.8,68.9,...,22.62,64.50,5.075,247.1,20.59,997.78815,0.020623,0.009368,13.71,26.27
1,08/02/2023,7:57:34,Container No Insulation,Temporary,1,3,1,3,22.0,69.1,...,22.88,64.13,5.238,247.0,24.80,997.79270,0.020280,0.009368,13.69,26.27
2,08/02/2023,8:20:29,Container No Insulation,Temporary,1,3,1,3,23.3,68.8,...,23.48,62.28,5.788,249.3,27.40,997.81710,0.020623,0.009368,13.71,24.54
3,08/02/2023,8:30:10,Container No Insulation,Temporary,1,1,1,3,22.7,61.2,...,23.61,61.91,5.188,240.8,24.80,997.83890,0.020280,0.009368,13.71,24.53
4,08/02/2023,8:46:21,Container No Insulation,Temporary,1,3,1,3,24.4,62.9,...,23.84,61.61,5.638,249.7,25.01,997.82200,0.020623,0.009368,13.71,24.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372079,07/11/2023,12:08:37,Mobile/Prefab,Temporary,1,65,2,0,23.7,48.4,...,19.33,54.47,4.412,232.9,18.44,1003.71680,0.742443,0.086734,13.63,23.97
372080,07/11/2023,12:09:14,Mobile/Prefab,Temporary,2,63,2,0,25.0,40.2,...,18.98,55.68,3.687,256.5,15.96,1003.72310,0.742442,0.086734,13.63,23.97
372081,07/11/2023,12:09:18,Mobile/Prefab,Temporary,1,66,2,0,23.8,48.9,...,18.98,55.68,3.687,256.5,15.96,1003.72310,0.742442,0.086734,13.63,23.97
372082,07/11/2023,12:09:51,Brick Single Story,Permanent,2,53,1,1,25.6,43.6,...,18.98,55.68,3.687,256.5,15.96,1003.72310,0.742442,0.086734,13.63,23.97
